<a href="https://colab.research.google.com/github/SRARNAB7/Python_Practice_Arnab/blob/main/Machine_Learning_Cls_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import statsmodels.api as sm

data1 = pd.read_excel("survey lung cancer.xlsx")
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309 entries, 0 to 308
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   GENDER                 309 non-null    object
 1   AGE                    309 non-null    int64 
 2   SMOKING                309 non-null    int64 
 3   YELLOW_FINGERS         309 non-null    int64 
 4   ANXIETY                309 non-null    int64 
 5   PEER_PRESSURE          309 non-null    int64 
 6   CHRONIC DISEASE        309 non-null    int64 
 7   FATIGUE                309 non-null    int64 
 8   ALLERGY                309 non-null    int64 
 9   WHEEZING               309 non-null    int64 
 10  ALCOHOL CONSUMING      309 non-null    int64 
 11  COUGHING               309 non-null    int64 
 12  SHORTNESS OF BREATH    309 non-null    int64 
 13  SWALLOWING DIFFICULTY  309 non-null    int64 
 14  CHEST PAIN             309 non-null    int64 
 15  LUNG_CANCER            

In [4]:
data2 = pd.read_csv("cancer_data.csv")
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   cancer_type             35 non-null     object 
 1   new_cases_number        35 non-null     int64  
 2   new_cases_rank          35 non-null     int64  
 3   new_case_percent        35 non-null     float64
 4   new_cases_cum_risk      35 non-null     float64
 5   Deaths Number           35 non-null     int64  
 6   death_rank              35 non-null     int64  
 7   death_percentage        35 non-null     float64
 8   Deaths_cum_risk         35 non-null     float64
 9   risk_of_prevalence_5yr  35 non-null     int64  
 10  per100k_5yr_prevalence  35 non-null     float64
dtypes: float64(5), int64(5), object(1)
memory usage: 3.1+ KB


In [5]:
data1.columns= [col.strip() for col in data1.columns]
data2.columns= [col.strip() for col in data2.columns]

In [6]:
#Creating age groups in servey data
data1['AGE_GROUP']= pd.cut(data1['AGE'],
                           bins = [0, 30, 45, 60, 100],
                           labels= ['<30', '30-45','45-60', '>60'])

data1['AGE_GROUP'].head()


,AGE_GROUP
0,>60
1,>60
2,45-60
3,>60
4,>60


In [9]:
#Aggregating Survey Data by Age Group

age_group_stats= data1.groupby('AGE_GROUP').agg({
    'LUNG_CANCER': lambda x:(x=='YES').mean(),
    'SMOKING':'mean',
    'ANXIETY':'mean'
}).reset_index()

age_group_stats.head()

<ipython-input-9-25787ce41809>:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  age_group_stats= data1.groupby('AGE_GROUP').agg({


,AGE_GROUP,LUNG_CANCER,SMOKING,ANXIETY
0,<30,0.000000,2.000000,1.000000
1,30-45,1.000000,1.500000,1.250000
2,45-60,0.847458,1.593220,1.457627
3,>60,0.892473,1.543011,1.532258


In [11]:
#Adding risk categories
age_group_stats['risk_category']= pd.qcut(age_group_stats['LUNG_CANCER'],
                                         q=3,
                                         labels=['Low','Medium','High'])
age_group_stats.head()

,AGE_GROUP,LUNG_CANCER,SMOKING,ANXIETY,risk_category
0,<30,0.000000,2.000000,1.000000,Low
1,30-45,1.000000,1.500000,1.250000,High
2,45-60,0.847458,1.593220,1.457627,Medium
3,>60,0.892473,1.543011,1.532258,Medium


In [17]:
#Merging the data

combined_data = pd.merge(age_group_stats,data2[['new_cases_number', 'Deaths Number']],
                         left_index= True, right_index= True, how='left')